In [ ]:
!tar zxvf ./drive/My\ Drive/Colab\ Notebooks/train.tgz


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Dropout, MaxPooling2D,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

training_size = 300000
batch_size = 128
epochs = 15
IMG_HEIGHT = 60
IMG_WIDTH = 28

CLASS_NAMES = np.array([])
for i in range(10):
    CLASS_NAMES =np.append(CLASS_NAMES,chr(ord("0")+i))
for i in range(26):
    CLASS_NAMES = np.append(CLASS_NAMES,chr(ord("A")+i))

#training dir / validation dir
train_dir_data01 = os.path.abspath(os.getcwd())+"/train/data01_train/"
#val_dir_data01 = os.path.abspath(os.getcwd())+"/dev/data01_dev"
num_train_data01 = len(os.listdir(train_dir_data01))
#num_val_data01 = len(os.listdir(val_dir_data01))
print("Train data count: " + str(num_train_data01))
#print("Validation data count: " + str(num_val_data01))

#store filename and label to a Dataframe 
import pandas as pd
data = pd.read_csv("./train/data01_train.csv")
for i in range(len(data)):
    data.iloc[i,1] = list(data.iloc[i,1])

#maybe there's an easier way to separate labels??
arr = np.zeros([6,50000],str)
for j in range(6):
    for i in range(len(data)):
        arr[j][i] = data.iloc[i,1][j]
data = data.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

data

In [ ]:
#training data 02
train_dir_data02 = os.path.abspath(os.getcwd())+"\\train\\data02_train\\"
data2 = pd.read_csv("train\\data02_train.csv")
for i in range(len(data2)):
    data2.iloc[i,1] = list(data2.iloc[i,1])
arr = np.zeros([6,50000],str)
for j in range(6):
    for i in range(len(data2)):
        arr[j][i] = data2.iloc[i,1][j]
data2 = data2.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

#to one hot
for i in range(6):
    data2.iloc[:,i+1] = data2.iloc[:,i+1].map(parse_label)

TL = tf.convert_to_tensor(list(data2.iloc[:,1].values))
TL2 = tf.convert_to_tensor(list(data2.iloc[:,2].values))
TL3 = tf.convert_to_tensor(list(data2.iloc[:,3].values))
TL4 = tf.convert_to_tensor(list(data2.iloc[:,4].values))
TL5 = tf.convert_to_tensor(list(data2.iloc[:,5].values))
TL6 = tf.convert_to_tensor(list(data2.iloc[:,6].values))

train_dir2 = tf.data.Dataset.list_files(train_dir_data02+'*.jpg',shuffle=False) 
train_data2 = train_dir2.map(lambda x: readimg_to_tensor(x))
train_label2 = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
train_data02=tf.data.Dataset.zip((train_data2,train_label2))

In [ ]:
#no crop function
def parse_label(strIN):
    return (strIN==CLASS_NAMES).astype(float)
def readimg_to_tensor(fn):
    a = tf.io.read_file((fn))
        #crop to [60,30,3]
    img = tf.io.decode_jpeg(a)
    return img

In [ ]:
#to one hot
for i in range(6):
    data.iloc[:,i+1] = data.iloc[:,i+1].map(parse_label)
data

In [ ]:
TL = tf.convert_to_tensor(list(data.iloc[:,1].values))
TL2 = tf.convert_to_tensor(list(data.iloc[:,2].values))
TL3 = tf.convert_to_tensor(list(data.iloc[:,3].values))
TL4 = tf.convert_to_tensor(list(data.iloc[:,4].values))
TL5 = tf.convert_to_tensor(list(data.iloc[:,5].values))
TL6 = tf.convert_to_tensor(list(data.iloc[:,6].values))

In [ ]:
#no crop
train_dir = tf.data.Dataset.list_files(train_dir_data01+'*.jpg',shuffle=False) 
train_data = train_dir.map(lambda x: readimg_to_tensor(x))
train_label = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
train_data01=tf.data.Dataset.zip((train_data,train_label))

In [ ]:
#validation data no crop
val_dir = tf.data.Dataset.list_files(val_dir_data01+'*.jpg',shuffle=False) 
val_data01 = val_dir.map(lambda x: readimg_to_tensor(x))

In [ ]:
#railway model
print("Creating CNN model...")
inn = Input((60, 200, 3))
out = inn
out = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu')(out)
out = Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Dropout(0.3)(out)
out = Conv2D(filters=256, kernel_size=(3, 3), padding='same',activation='relu')(out)
out = Conv2D(filters=256, kernel_size=(3, 3), activation='relu')(out)
out = BatchNormalization()(out)
out = MaxPooling2D(pool_size=(2, 2))(out)
out = Flatten()(out)
sep = Dropout(0.3)(out)
dig1 = Dense(36, name='digit1', activation='softmax')(sep)
dig2 = Dense(36, name='digit2', activation='softmax')(sep)
dig3 = Dense(36, name='digit3', activation='softmax')(sep)
dig4 = Dense(36, name='digit4', activation='softmax')(sep)
dig5 = Dense(36, name='digit5', activation='softmax')(sep)
dig6 = Dense(36, name='digit6', activation='softmax')(sep)
model = tf.keras.models.Model(inputs=inn, outputs=[dig1, dig2,dig3,dig4,dig5,dig6])
model.compile(
    loss=[
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
    ],
    optimizer='adam', 
    metrics=['accuracy'])
model.summary()

In [ ]:
#number of element
num_elements = tf.data.experimental.cardinality(train_data01).numpy()
num_elements

In [ ]:
model = tf.keras.models.load_model("/content/data01_20200521.h5")

In [ ]:
epochs=30
batch_size=128
big_batch_size=5000
train_data_gen=iter(train_data01.repeat().batch(big_batch_size))

for i in range(int(num_elements/big_batch_size)):
   img , lb = next(train_data_gen)
   history = model.fit(
       img,
       lb,
       batch_size=batch_size,
       validation_split=0.1,
       shuffle=True,
       use_multiprocessing=True,
       epochs=epochs,
       verbose=2
       )


In [ ]:
def parseResult(RS):
    for j in range(len(RS[0])):
        PredChr=''
        for i in range(6):
            PredChr+= CLASS_NAMES[(np.where(RS[i][j] == RS[i][j].max())[0][0])]
        if j == 0:
            LB = [PredChr]
        else:
            LB.append(PredChr)      
    return LB
        

In [ ]:
#output to CSV
VDlabel = np.array([])
img = next(iter(val_data01.batch(10000)))
result = model.predict(img,use_multiprocessing = True)


In [ ]:
val_dir = tf.data.Dataset.list_files(val_dir_data01+'*.jpg',shuffle=False) 
test = parseResult(result)
val_file_name = pd.read_csv("./dev/data01_dev.csv")
val_file_name.join(pd.DataFrame(test,columns=["code"])).to_csv("data01_dev.csv",index = False)

In [ ]:
model.save('data01_20200521_46epochs.h5')